# Joining Hurricane and Housing Dataframes 

In [58]:
#Importing libraries needed
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import pyplot
%matplotlib inline
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Joining Housing Values with Hurricanes 
In order to do logistic regression on our data we need to join the datasets. We will use the join method joining the hurricane dataset into the housing dataset. 

Documentation can be found here: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html

## Joining 1 year before and after hurricane 

In [59]:
#opening dataframes 
hurricane1 = pd.read_csv(r'data\hurricane_clean.csv')
housing1 =  pd.read_csv(r'data\housing_1year.csv')

In [60]:
#setting the index to city so that we use .join()
hurricane1.set_index(['City'], inplace = True)
housing1.set_index(['City'], inplace = True)

In [61]:
#joining the housing dataframe into the hurricane dataframe 
df1 = hurricane1.join(housing1, how='inner')

In [62]:
#reseting the index
df1.reset_index(inplace = True)
df1.head()

,City,DATE,AWND,WSF2,WSF5,HurricaneName,SizeRank,bc,ac,bd,...,percentc,percentd,percentma,percentir,percentmi,boolc,boold,boolma,boolir,boolmi
0,Apalachicola,8/14/2004,5.82,13.0,15.0,1,12877,84666.67437,128305.1212,100185.4978,...,51.541468,48.238795,13.879717,25.2258,20.665211,1,0,0,1,1
1,Apalachicola,7/10/2005,19.46,30.0,34.9,2,12877,84666.67437,128305.1212,100185.4978,...,51.541468,48.238795,13.879717,25.2258,20.665211,1,0,0,1,1
2,Apalachicola,7/11/2005,17.00,32.0,38.0,2,12877,84666.67437,128305.1212,100185.4978,...,51.541468,48.238795,13.879717,25.2258,20.665211,1,0,0,1,1
3,Apalachicola,10/7/2016,10.74,21.9,27.1,3,12877,84666.67437,128305.1212,100185.4978,...,51.541468,48.238795,13.879717,25.2258,20.665211,1,0,0,1,1
4,Apalachicola,10/8/2016,8.05,15.0,21.9,3,12877,84666.67437,128305.1212,100185.4978,...,51.541468,48.238795,13.879717,25.2258,20.665211,1,0,0,1,1


In [63]:
#saving the h1 dataframe
df1.to_csv(r'data\oneyear.csv', index=False)

## Joining 6 months before and after hurricane 

In [64]:
#opening the dataframes 
hurricane6 = pd.read_csv(r'data\hurricane_clean.csv')
housing6 =  pd.read_csv(r'data\housing_6months.csv')

In [65]:
#setting the index to city so that we use .join()
hurricane6.set_index(['City'], inplace = True)
housing6.set_index(['City'], inplace = True)

In [66]:
#joining the housing dataframe into the hurricane dataframe 
df6 = hurricane6.join(housing6, how='inner')

In [67]:
#reseting the index
df6.reset_index(inplace = True)
df6.head()

,City,DATE,AWND,WSF2,WSF5,HurricaneName,SizeRank,bc,ac,bd,...,percentc,percentd,percentma,percentir,percentmi,boolc,boold,boolma,boolir,boolmi
0,Apalachicola,8/14/2004,5.82,13.0,15.0,1,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
1,Apalachicola,7/10/2005,19.46,30.0,34.9,2,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
2,Apalachicola,7/11/2005,17.00,32.0,38.0,2,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
3,Apalachicola,10/7/2016,10.74,21.9,27.1,3,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1
4,Apalachicola,10/8/2016,8.05,15.0,21.9,3,12877,91915.39335,114408.1491,112287.8968,...,24.471152,24.881677,3.115506,11.045477,11.724797,1,0,0,1,1


In [68]:
#saving the h6 dataframe
df6.to_csv(r'data\sixmonths.csv', index=False)

## Joining 3 months before and after hurricane 

In [69]:
#opening the dataframes 
hurricane3 = pd.read_csv(r'data\hurricane_clean.csv')
housing3 =  pd.read_csv(r'data\housing_3months.csv')

In [70]:
#setting the index to city so that we use .join()
hurricane3.set_index(['City'], inplace = True)
housing3.set_index(['City'], inplace = True)

In [71]:
#joining the housing dataframe into the hurricane dataframe 
df3 = hurricane3.join(housing3, how='inner')

In [72]:
#reseting the index
df3.reset_index(inplace = True)
df3.head()

,City,DATE,AWND,WSF2,WSF5,HurricaneName,SizeRank,bc,ac,bd,...,percentc,percentd,percentma,percentir,percentmi,boolc,boold,boolma,boolir,boolmi
0,Apalachicola,8/14/2004,5.82,13.0,15.0,1,12877,97342.4766,107842.5207,118373.8681,...,10.786703,13.359012,2.213214,6.87801,7.389938,0,0,0,1,1
1,Apalachicola,7/10/2005,19.46,30.0,34.9,2,12877,97342.4766,107842.5207,118373.8681,...,10.786703,13.359012,2.213214,6.87801,7.389938,0,0,0,1,1
2,Apalachicola,7/11/2005,17.00,32.0,38.0,2,12877,97342.4766,107842.5207,118373.8681,...,10.786703,13.359012,2.213214,6.87801,7.389938,0,0,0,1,1
3,Apalachicola,10/7/2016,10.74,21.9,27.1,3,12877,97342.4766,107842.5207,118373.8681,...,10.786703,13.359012,2.213214,6.87801,7.389938,0,0,0,1,1
4,Apalachicola,10/8/2016,8.05,15.0,21.9,3,12877,97342.4766,107842.5207,118373.8681,...,10.786703,13.359012,2.213214,6.87801,7.389938,0,0,0,1,1


In [73]:
#saving the h3 dataframe
df3.to_csv(r'data\threemonths.csv', index=False)

## Joining top tier housing before and after hurricane 

In [74]:
#opening dataframes
hurricanet = pd.read_csv(r'data\hurricane_clean.csv')
housingt =  pd.read_csv(r'data\toptier1year.csv')

In [75]:
#setting the index to city so that we use .join()
hurricanet.set_index(['City'], inplace = True)
housingt.set_index(['City'], inplace = True)

In [76]:
#joining the housing dataframe into the hurricane dataframe 
dft = hurricanet.join(housingt, how='inner')

In [77]:
#saving the dft dataframe
dft.reset_index(inplace = True)
dft.head()

,City,DATE,AWND,WSF2,WSF5,HurricaneName,SizeRank,bc,ac,bd,...,percentc,percentd,percentma,percentir,percentmi,boolc,boold,boolma,boolir,boolmi
0,Apalachicola,8/14/2004,5.82,13.0,15.0,1,12877,205983.955,309761.0651,241873.3344,...,50.381162,48.643917,8.237653,18.451984,15.893959,1,0,0,1,1
1,Apalachicola,7/10/2005,19.46,30.0,34.9,2,12877,205983.955,309761.0651,241873.3344,...,50.381162,48.643917,8.237653,18.451984,15.893959,1,0,0,1,1
2,Apalachicola,7/11/2005,17.00,32.0,38.0,2,12877,205983.955,309761.0651,241873.3344,...,50.381162,48.643917,8.237653,18.451984,15.893959,1,0,0,1,1
3,Apalachicola,10/7/2016,10.74,21.9,27.1,3,12877,205983.955,309761.0651,241873.3344,...,50.381162,48.643917,8.237653,18.451984,15.893959,1,0,0,1,1
4,Apalachicola,10/8/2016,8.05,15.0,21.9,3,12877,205983.955,309761.0651,241873.3344,...,50.381162,48.643917,8.237653,18.451984,15.893959,1,0,0,1,1


In [78]:
#saving the dft dataframe
dft.to_csv(r'data\top.csv', index=False)

## Joining bottom tier before and after hurricane 

In [79]:
#opening dataframes
hurricaneb = pd.read_csv(r'data\hurricane_clean.csv')
housingb =  pd.read_csv(r'data\bottomtier1year.csv')

In [80]:
#setting the index to city so that we use .join()
hurricaneb.set_index(['City'], inplace = True)
housingb.set_index(['City'], inplace = True)

In [81]:
#joining the housing dataframe into the hurricane dataframe 
dfb = hurricaneb.join(housingb, how='inner')

In [82]:
#saving the dft dataframe
dfb.reset_index(inplace = True)
dfb.head()

,City,DATE,AWND,WSF2,WSF5,HurricaneName,SizeRank,bc,ac,bd,...,percentc,percentd,percentma,percentir,percentmi,boolc,boold,boolma,boolir,boolmi
0,Apalachicola,8/14/2004,5.82,13.0,15.0,1,12877,30464.93986,46467.43038,35905.97635,...,52.527563,51.289271,24.954656,49.808378,38.192255,0,0,0,1,1
1,Apalachicola,7/10/2005,19.46,30.0,34.9,2,12877,30464.93986,46467.43038,35905.97635,...,52.527563,51.289271,24.954656,49.808378,38.192255,0,0,0,1,1
2,Apalachicola,7/11/2005,17.00,32.0,38.0,2,12877,30464.93986,46467.43038,35905.97635,...,52.527563,51.289271,24.954656,49.808378,38.192255,0,0,0,1,1
3,Apalachicola,10/7/2016,10.74,21.9,27.1,3,12877,30464.93986,46467.43038,35905.97635,...,52.527563,51.289271,24.954656,49.808378,38.192255,0,0,0,1,1
4,Apalachicola,10/8/2016,8.05,15.0,21.9,3,12877,30464.93986,46467.43038,35905.97635,...,52.527563,51.289271,24.954656,49.808378,38.192255,0,0,0,1,1


In [83]:
#saving the dfb dataframe
dfb.to_csv(r'data\bottom.csv', index=False)